In [2]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
#from sklearn.neural_network import MLPClassifier

def log_progress(sequence, every=None, size=None):
    from ipywidgets import IntProgress, HTML, VBox
    from IPython.display import display

    is_iterator = False
    if size is None:
        try:
            size = len(sequence)
        except TypeError:
            is_iterator = True
    if size is not None:
        if every is None:
            if size <= 200:
                every = 1
            else:
                every = size / 200     # every 0.5%
    else:
        assert every is not None, 'sequence is iterator, set every'

    if is_iterator:
        progress = IntProgress(min=0, max=1, value=1)
        progress.bar_style = 'info'
    else:
        progress = IntProgress(min=0, max=size, value=0)
    label = HTML()
    box = VBox(children=[label, progress])
    display(box)

    index = 0
    try:
        for index, record in enumerate(sequence, 1):
            if index == 1 or index % every == 0:
                if is_iterator:
                    label.value = '{index} / ?'.format(index=index)
                else:
                    progress.value = index
                    label.value = u'{index} / {size}'.format(
                        index=index,
                        size=size
                    )
            yield record
    except:
        progress.bar_style = 'danger'
        raise
    else:
        progress.bar_style = 'success'
        progress.value = index
        label.value = str(index or '?')

In [3]:
location = ''
events = pd.DataFrame.from_csv(location+"events_modifyed.csv",index_col=None)
structure = pd.DataFrame.from_csv(location+"structure.csv",index_col=None)
targets = pd.DataFrame.from_csv(location+"targets.csv",index_col=None)
events_test = pd.DataFrame.from_csv(location+"events_test_modifyed.csv",index_col=None)

In [4]:
events.sort_values(by='time', inplace=True)

In [ ]:
for i, row in events_test.iterrows():
    if events_test.loc[i,'action'] == 'discovered':
        events_test.set_value(i, 'action', 1)
    if events_test.loc[i, 'action'] == 'passed':
        events_test.set_value(i, 'action', 2)
    if events_test.loc[i, 'action'] == 'viewed':
        events_test.set_value(i, 'action', 3)
    if events_test.loc[i, 'action'] == 'started_attempt':
        events_test.set_value(i, 'action', 4)
    if events_test.loc[i, 'step_type'] == 'choice':
        events_test.set_value(i, 'step_type', 1)
    if events_test.loc[i, 'step_type'] == 'code':
        events_test.set_value(i, 'step_type', 2)
    if events_test.loc[i, 'step_type'] == 'number':
        events_test.set_value(i, 'step_type', 3)
    if events_test.loc[i, 'step_type'] == 'string':
        events_test.set_value(i, 'step_type', 4)
    if events_test.loc[i, 'step_type'] == 'text':
        events_test.set_value(i, 'step_type', 5)
    if events_test.loc[i, 'step_type'] == 'video':
        events_test.set_value(i, 'step_type', 6)
        

In [91]:
events.to_csv('events_modifyed.csv')

In [4]:
structure.sort_values(['module_position','lesson_position', 'step_position'], inplace=True)

In [5]:
step_id_position = structure.step_id.values.tolist()

In [6]:
X = [] #по каждому юзеру имеем век. где параметры : id, набран счет, популяр.рубрика
for us_id in log_progress(set(events.user_id.tolist())):
    temp_us_data = events[events.user_id == us_id]
    temp_us_to_app_vec = []
    
    temp_us_to_app_vec.append(us_id)
    
    temp_us_to_app_vec.append(temp_us_data.step_cost.sum())
    temp_us_to_app_vec.append(temp_us_data.step_cost.mean())
    temp_us_to_app_vec.extend(temp_us_data.step_cost.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.step_type.max())
    temp_us_to_app_vec.append(temp_us_data.step_type.min())
    temp_us_to_app_vec.extend(temp_us_data.step_type.describe().tolist())
    temp_us_to_app_vec.extend([len(temp_us_data[temp_us_data.step_type == i]) for i in range(1,7)])
    
    temp_us_to_app_vec.append(temp_us_data.time.mean())
    temp_us_to_app_vec.append(temp_us_data.time.max() - temp_us_data.time.min())
    temp_us_to_app_vec.extend(temp_us_data.time.describe().values.tolist())


    temp_us_to_app_vec.extend(temp_us_data.action.describe().tolist())
    temp_us_to_app_vec.extend([len(temp_us_data[temp_us_data.action == i]) for i in range(1,5)])
    
    pos_frame = pd.DataFrame([step_id_position.index(elem) for elem in temp_us_data.step_id])
    for elem in pos_frame.describe().values.tolist():
        temp_us_to_app_vec.append(elem[0])
    
    
    X.append(temp_us_to_app_vec)
Y = []

for elem in log_progress(X):
    Y.append([targets[targets.user_id == elem[0]].passed.values[0]])

In [7]:
X_test = [] #по каждому юзеру имеем век. где параметры : id, набран счет, популяр.рубрика
for us_id in log_progress(set(events_test.user_id.tolist())):
    temp_us_data = events_test[events_test.user_id == us_id]
    temp_us_to_app_vec = []
    
    temp_us_to_app_vec.append(us_id)
    
    temp_us_to_app_vec.append(temp_us_data.step_cost.sum())
    temp_us_to_app_vec.append(temp_us_data.step_cost.mean())
    temp_us_to_app_vec.extend(temp_us_data.step_cost.describe().tolist())
    
    temp_us_to_app_vec.append(temp_us_data.step_type.max())
    temp_us_to_app_vec.append(temp_us_data.step_type.min())
    temp_us_to_app_vec.extend(temp_us_data.step_type.describe().tolist())
    temp_us_to_app_vec.extend([len(temp_us_data[temp_us_data.step_type == i]) for i in range(1,7)])
    
    temp_us_to_app_vec.append(temp_us_data.time.mean())
    temp_us_to_app_vec.append(temp_us_data.time.max() - temp_us_data.time.min())
    temp_us_to_app_vec.extend(temp_us_data.time.describe().values.tolist())


    temp_us_to_app_vec.extend(temp_us_data.action.describe().tolist())
    temp_us_to_app_vec.extend([len(temp_us_data[temp_us_data.action == i]) for i in range(1,5)])
    
    pos_frame = pd.DataFrame([step_id_position.index(elem) for elem in temp_us_data.step_id])
    for elem in pos_frame.describe().values.tolist():
        temp_us_to_app_vec.append(elem[0])
    
    
    X_test.append(temp_us_to_app_vec)

In [8]:
X_new = pd.DataFrame(X).fillna(method='ffill')
X_test = pd.DataFrame(X_test).fillna(method='ffill')

X_new = X_new.as_matrix()

Y = np.ravel(Y)
Y = Y.astype(np.int32)

X_new = X_new.astype(np.float32)

In [ ]:
from sklearn.manifold import TSNE

model = TSNE()

plt.plot(model.fit_transform(X_new[:100]))


In [50]:
#model = DummyClassifier( constant=0)
from sklearn.ensemble import ExtraTreesClassifier
model =  ExtraTreesClassifier(verbose=1, n_jobs = -1, n_estimators=1000)

In [166]:
from sklearn.linear_model import SGDClassifier
model = SGDClassifier( n_jobs=-1, verbose=True, n_iter=1000)

In [315]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier(n_jobs=-1, n_neighbors=2)

In [136]:
from sklearn.neural_network import MLPClassifier
model = MLPClassifier(hidden_layer_sizes=(98,47,2), verbose=True)

In [10]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import PolynomialFeatures

model = RandomForestClassifier(verbose=1, n_jobs = -1, n_estimators = 10000)

In [80]:
from xgboost import XGBClassifier

model = XGBClassifier(n_estimators=1000, silent=False, max_depth= 10)

In [28]:
from sklearn.cross_validation import train_test_split
from sklearn.metrics import f1_score
#Поделим нашу выборку на train и validation
Xtr,Xval,Ytr,Yval = train_test_split(X_new,Y,test_size=0.2,random_state=128)

Xtr = PolynomialFeatures(3).fit_transform(Xtr)
Xval = PolynomialFeatures(3).fit_transform(Xval)

In [29]:
#Обучим модель
model.fit(Xtr, Ytr)

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.7s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   26.8s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:  2.9min
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:  4.1min
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  5.6min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  7.3min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  9.3min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed: 11.4min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed: 13.8min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed: 16.5min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed: 19.5min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed: 22.6min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed: 23.1min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10000, n_jobs=-1, oob_score=False,
            random_state=None, verbose=1, warm_start=False)

In [94]:
optim = []
pred_prob = []
for i in log_progress([32, 64, 128, 256, 512, 16, 8, 100, 50, 33]):
    Xtr,Xval,Ytr,Yval = train_test_split(X_new,Y,test_size=0.2,random_state=i)
    Xtr = PolynomialFeatures(2).fit_transform(Xtr)
    Xval = PolynomialFeatures(2).fit_transform(Xval)
    optim.append(Yval)
    model.fit(Xtr, Ytr)
    pred_prob.append(model.predict_proba(Xval))
    

[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.2s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:    6.5s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   13.5s
[Parallel(n_jobs=-1)]: Done 784 tasks      | elapsed:   23.4s
[Parallel(n_jobs=-1)]: Done 1234 tasks      | elapsed:   35.5s
[Parallel(n_jobs=-1)]: Done 1784 tasks      | elapsed:   50.2s
[Parallel(n_jobs=-1)]: Done 2434 tasks      | elapsed:  1.1min
[Parallel(n_jobs=-1)]: Done 3184 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 4034 tasks      | elapsed:  1.8min
[Parallel(n_jobs=-1)]: Done 4984 tasks      | elapsed:  2.3min
[Parallel(n_jobs=-1)]: Done 6034 tasks      | elapsed:  2.7min
[Parallel(n_jobs=-1)]: Done 7184 tasks      | elapsed:  3.2min
[Parallel(n_jobs=-1)]: Done 8434 tasks      | elapsed:  3.8min
[Parallel(n_jobs=-1)]: Done 9784 tasks      | elapsed:  4.4min
[Parallel(n_jobs=-1)]: Done 10000 out of 10000 | elapsed:  4.5min finished
[Parallel(n_jobs=8)]: Done  34 tasks      | ela

In [95]:
max_score = 0
best_eps = 0
for k in range(100):
    eps = k * 0.01
    predict_eps = []
    for elem in pred_prob:
        temp_pred = []
        for row in elem:
            if row[1] > eps:
                temp_pred.append(1)
            else:
                temp_pred.append(0)
        predict_eps.append(temp_pred)
    f1_score_test = []
    for i  in range(len(predict_eps)):
        f1_score_test.append(f1_score(optim[i], predict_eps[i]))
    if np.array(f1_score_test).mean() > max_score:
        max_score = np.array(f1_score_test).mean()
        best_eps = eps
print(max_score)
print(eps)

/home/axcel/anaconda3/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


0.571808864454
0.99


In [79]:
model.predict(Xval)[0]

0

In [30]:
pred_prob = model.predict_proba(Xval)


f1_score_temp = []

for i in range(10000):
    ans = []
    eps = i * 0.0001
    for elem in pred_prob:
        if elem[1] > eps:
            ans.append(1)
        else:
            ans.append(0)
    f1_score_temp.append(f1_score(Yval, ans))


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    2.3s
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed:    2.8s
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed:    3.3s
[Parallel(n_jobs=8)]: Done 8434 tasks      | elapsed:    3.9s
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed:    4.5s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:    4.6s finished
/home/axcel/anaconda3/lib/python3.5/site-packages/sklearn/metr

In [31]:
maxnum = 0
pos = 0
for i in range(len(f1_score_temp)):
    if maxnum < f1_score_temp[i]:
        maxnum = f1_score_temp[i]
        pos = i
print(maxnum)
print(pos)

0.613545816733
4033


In [33]:
ans = []
pred_prob = model.predict_proba(PolynomialFeatures(3).fit_transform(X_test))

for elem in pred_prob:
    if elem[1] > 0.4:
        ans.append(1)
    else:
        ans.append(0)


[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.3s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    1.5s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    2.4s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    3.0s
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed:    3.5s
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed:    4.2s
[Parallel(n_jobs=8)]: Done 8434 tasks      | elapsed:    4.9s
[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed:    5.6s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:    5.7s finished


In [142]:
f1_score(Yval, ans)

0.60000000000000009

In [105]:
#Оценим метрику на validation
print(f1_score(Yval, model.predict(Xval)))

[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 784 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 1234 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 1784 tasks      | elapsed:    0.5s
[Parallel(n_jobs=8)]: Done 2434 tasks      | elapsed:    0.7s
[Parallel(n_jobs=8)]: Done 3184 tasks      | elapsed:    0.9s
[Parallel(n_jobs=8)]: Done 4034 tasks      | elapsed:    1.1s
[Parallel(n_jobs=8)]: Done 4984 tasks      | elapsed:    1.3s
[Parallel(n_jobs=8)]: Done 6034 tasks      | elapsed:    1.6s
[Parallel(n_jobs=8)]: Done 7184 tasks      | elapsed:    1.9s
[Parallel(n_jobs=8)]: Done 8434 tasks      | elapsed:    2.2s


0.482352941176


[Parallel(n_jobs=8)]: Done 9784 tasks      | elapsed:    2.6s
[Parallel(n_jobs=8)]: Done 10000 out of 10000 | elapsed:    2.6s finished


In [39]:
ind = X_test[0].tolist()

In [41]:
#Предскажем результат и запишем его в .csv

def create_submission(X, name):
    np.savetxt('%s.csv'%name, X, delimiter=',', fmt="%d", header='user_id,passed', comments='')
    
#ans = model.predict(PolynomialFeatures(3).fit_transform(X_test))
#ans = model.predict(X_test)

result = np.concatenate((np.asarray(ind, dtype=int).reshape(-1, 1), np.asarray(ans, dtype=int).reshape(-1, 1)), axis=1)
create_submission(result, '0.61_lead_polfeat3_randforest')

In [11]:
from tpot import TPOTClassifier
from sklearn.cross_validation import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_new[:10 ** 4], Y[:10**4],
                                                 test_size=0.20)

tpot = TPOTClassifier(generations=5, population_size=20, verbosity=3, scoring='f1')
tpot.fit(X_train, y_train)
print(tpot.score(X_test, y_test))
tpot.export('tpot_goto_pipeline.py')

GP Progress:  17%|█▋        | 20/120 [00:00<10:42,  6.43s/pipeline]

Generation 1 - Current Pareto front scores:
1	0.46260557993158863	XGBClassifier(input_matrix, 30, 8, 37.0, 0.70999999999999996)
2	0.474686629078247	XGBClassifier(GaussianNB(input_matrix), 8, 19, 0.5, 40.0)



GP Progress:  31%|███       | 37/120 [00:00<14:20, 10.36s/pipeline]

Generation 2 - Current Pareto front scores:
1	0.46260557993158863	XGBClassifier(input_matrix, 30, 8, 37.0, 0.70999999999999996)
2	0.474686629078247	XGBClassifier(GaussianNB(input_matrix), 8, 19, 0.5, 40.0)



GP Progress:  48%|████▊     | 58/120 [00:00<02:58,  2.88s/pipeline]

...

GP Progress:  65%|██████▌   | 78/120 [00:00<02:01,  2.90s/pipeline]

Generation 4 - Current Pareto front scores:
1	0.4833657289341599	XGBClassifier(input_matrix, 13, 28, 0.17999999999999999, 0.60999999999999999)



GP Progress:  82%|████████▏ | 98/120 [00:00<01:33,  4.26s/pipeline]

Generation 5 - Current Pareto front scores:
1	0.4833657289341599	XGBClassifier(input_matrix, 13, 28, 0.17999999999999999, 0.60999999999999999)



0.501992031873


In [362]:
def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.arange(len(inputs))
        np.random.shuffle(indices)
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt]
        
def multilabel_objective(predictions, targets):
    epsilon = np.float32(1.0e-6)
    one = np.float32(1.0)
    pred = T.clip(predictions, epsilon, one - epsilon)
    return -T.sum(targets * T.log(pred) + (one - targets) * T.log(one - pred), axis=1)

def fmeasure(prediction, target, alpha=0.5):
    """Return the approximated f-measure loss between the `target` and
    the `prediction`. This is an overall loss, not a sample-wise one,
    that is, the loss is computed for all the samples at once.
    Jansche, Martin. "Maximum expected F-measure training of logistic
    regression models." Proceedings of the conference on Human Language
    Technology and Empirical Methods in Natural Language Processing.
    Association for Computational Linguistics, 2005.
    Parameters
    ----------
    target : Theano variable
        An array of arbitrary shape representing representing the targets.
    prediction : Theano variable
        An array of arbitrary shape representing representing the predictions.
    Returns
    -------
    res : Theano variable
        An array of the same columns as ``target`` and ``prediction``
        representing the overall f-measure loss."""
    n_pos = target.sum(axis=0)
    m_pos = prediction.sum(axis=0)
    true_positives_approx = target*prediction
    A = true_positives_approx.sum(axis=0)
    return A/(alpha*n_pos+(1-alpha)*m_pos)

In [292]:
import lasagne
import theano
import theano.tensor as T

input_X = T.matrix("X")

#input dimention (None means "Arbitrary")
input_shape = [None, 57]

target_y = T.vector("target Y integer",dtype='int64')

In [374]:
#Input layer (auxilary)
input_layer = lasagne.layers.InputLayer(shape = input_shape, input_var=input_X)

#fully connected layer, that takes input layer and applies 50 neurons to it.
# nonlinearity here is sigmoid as in logistic regression
# you can give a name to each layer (optional)


dense_1 = lasagne.layers.DenseLayer(input_layer, num_units=100,
                                   nonlinearity = lasagne.nonlinearities.rectify,
                                   name = "hidden_dense_layer - 1", W=lasagne.init.GlorotUniform())

dense_2 = lasagne.layers.DenseLayer(dense_1, num_units=16, nonlinearity=lasagne.nonlinearities.rectify,
                                   W=lasagne.init.GlorotUniform())


#fully connected output layer that takes dense_1 as input and has 10 neurons (1 for each digit)
#We use softmax nonlinearity to make probabilities add up to 1
dense_output = lasagne.layers.DenseLayer(dense_2, num_units = 1 ,
                                        nonlinearity = lasagne.nonlinearities.sigmoid,
                                        name='output', W=lasagne.init.Normal())


#network prediction (theano-transformation)
y_predicted = lasagne.layers.get_output(dense_output)

#all network weights (shared variables)
all_weights = lasagne.layers.get_all_params(dense_output, trainable=True)

print(all_weights)

#Mean categorical crossentropy as a loss function - similar to logistic loss but for multiclass targets
loss = fmeasure(y_predicted, target_y).mean()

#prediction accuracy (WITH dropout)

accuracy = fmeasure(y_predicted, target_y).mean()

#This function computes gradient AND composes weight updates just like you did earlier
updates_sgd = lasagne.updates.adam(loss, all_weights)

#function that computes loss and updates weights
train_fun = theano.function([input_X,target_y],[loss,accuracy], updates= updates_sgd)


#deterministic prediciton (without dropout)
y_predicted_det = lasagne.layers.get_output(dense_output, deterministic=True)

#prediction accuracy (without dropout)
accuracy_det = lasagne.objectives.categorical_accuracy(y_predicted_det,target_y).mean()

#function that just computes accuracy without dropout/noize -- for evaluation purposes
accuracy_fun = theano.function([input_X,target_y],accuracy_det)

[hidden_dense_layer - 1.W, hidden_dense_layer - 1.b, W, b, output.W, output.b]


In [375]:
import time

num_epochs = 5 #amount of passes through the data
            
batch_size = 1 #number of samples processed at each function call

for epoch in range(num_epochs):
    # In each epoch, we do a full pass over the training data:
    train_err = 0
    train_acc = 0
    train_batches = 0
    start_time = time.time()
    for batch in iterate_minibatches(Xtr, Ytr,batch_size):
        inputs, targets = batch
        train_err_batch, train_acc_batch= train_fun(inputs, targets)
        train_err += train_err_batch
        train_acc += train_acc_batch
        train_batches += 1

    # And a full pass over the validation data:
    val_acc = 0
    val_batches = 0
    for batch in iterate_minibatches(Xval, Yval, batch_size):
        inputs, targets = batch
        val_acc += accuracy_fun(inputs, targets)
        val_batches += 1

    
    # Then we print the results for this epoch:
    print("Epoch {} of {} took {:.3f}s".format(
        epoch + 1, num_epochs, time.time() - start_time))

    print("  training loss (in-iteration):\t\t{:.6f}".format(train_err / train_batches))
    print("  train accuracy:\t\t{:.2f} %".format(
        train_acc / train_batches * 100))
    print("  validation accuracy:\t\t{:.2f} %".format(
        val_acc / val_batches * 100))

Epoch 1 of 5 took 10.449s
  training loss (in-iteration):		0.040752
  train accuracy:		4.08 %
  validation accuracy:		96.48 %
Epoch 2 of 5 took 10.031s
  training loss (in-iteration):		0.040752
  train accuracy:		4.08 %
  validation accuracy:		96.48 %
Epoch 3 of 5 took 10.428s
  training loss (in-iteration):		0.040752
  train accuracy:		4.08 %
  validation accuracy:		96.48 %
Epoch 4 of 5 took 10.007s
  training loss (in-iteration):		0.040752
  train accuracy:		4.08 %
  validation accuracy:		96.48 %
Epoch 5 of 5 took 9.955s
  training loss (in-iteration):		0.040752
  train accuracy:		4.08 %
  validation accuracy:		96.48 %


In [370]:
out = lasagne.layers.get_output(dense_output, inputs=Xval)

norm_inp = lasagne.layers.get_output(dense_2, Xval)

out = out.eval()

out_ans = []

#f1_score(np.ravel(out),Yval)
fmeasure(np.ravel(out),Yval)

nan

In [352]:
np.ravel(out[:10])

array([ 1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.,  1.], dtype=float32)

In [321]:
norm_inp.eval()[0]

array([ 0.4227671 ,  0.        ,  0.        ,  0.08162348,  0.38785744,
        0.        ,  1.74059975,  0.13581587,  2.30880523,  0.25078034,
        0.        ,  1.21350598,  0.05336346,  0.24761648,  0.45958593,  0.        ], dtype=float32)

In [288]:
norm_inp.eval()[0]

array([ nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,  nan,
        nan,  nan,  nan,  nan,  nan,  nan], dtype=float32)

In [100]:
out[:30]

array([[ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,  0.16731696],
       [ 0.83268303,

In [126]:
type(Xval)

numpy.ndarray

In [113]:
out = 

Shape.0

In [82]:
test_acc = 0
test_batches = 0
for batch in iterate_minibatches(Xval, Yval, 500):
    inputs, targets = batch
    acc = accuracy_fun(inputs, targets)
    test_acc += acc
    test_batches += 1
print("Final results:")
print("  test accuracy:\t\t{:.2f} %".format(
    test_acc / test_batches * 100))

if test_acc / test_batches * 100 > 95:
    print ("Double-check, than consider applying for NIPS'17. SRSly.")
elif test_acc / test_batches * 100 > 90:
    print ("U'r freakin' amazin'!")
elif test_acc / test_batches * 100 > 80:
    print ("Achievement unlocked: 110lvl Warlock!")
elif test_acc / test_batches * 100 > 70:
    print ("Achievement unlocked: 80lvl Warlock!")
elif test_acc / test_batches * 100 > 50:
    print ("Achievement unlocked: 60lvl Warlock!")
else:
    print ("We need more magic!")

Final results:
  test accuracy:		96.43 %
Double-check, than consider applying for NIPS'17. SRSly.


In [142]:
from IPython.display import IFrame
from theano.d3viz.d3viz import d3viz
import theano.d3viz as d3v
d3v.d3viz(y_predicted, 'examples/mlp.html')
IFrame('examples/mlp.html', width=1000, height=700)